In [19]:
# 各行動での(行動後の)現在地と向きを記録しておく
# 命令を変更せずに全ての行動が終わった時の現在地を記録しておく

# 各行動について，(行動後の)現在地と向きに対する全命令後の座標の距離(向いている方向に何マス，右に何マス)を計算する
# 各行動を別の行動に変えた時に，目的地に到達するかどうかを判定する
# 到達する場合は，その行動以降をシミュレーションして，障害物にぶつかるかどうかを判定する
# 障害物にぶつからない場合は，その行動を実行するように命令して終了
# 障害物にぶつかる場合，到達しない場合は，次の行動について同様のことを行う

from json import dumps, loads
from typing import List

def find_correct_path(instructions: str, target: List[int], obstacles: List[List[int]]) -> str:
    # 目的地のx,y座標
    tx,ty = target
    # 障害物の座標
    obstacles = set([tuple(o) for o in obstacles])
    # R,L,U,D，各方向を向いている時のFの座標の変化
    d_dict = {"R":(1,0),"L":(-1,0),"U":(0,1),"D":(0,-1)}
    # R,L,U,D，各方向を向いている時のRの向きの変化
    r_dict = {"R":"D","D":"L","L":"U","U":"R"}
    # R,L,U,D，各方向を向いている時のLの向きの変化
    l_dict = {"R":"U","U":"L","L":"D","D":"R"}
    # ind番目の命令をinstに置き換える
    ind = 0
    inst = ""
    # 各行動での(行動後の)現在地と向き (x座標,y座標,方向) xy_d[i]にinstructions[i]を実行するとxy_d[i+1]になる
    xy_d = [tuple([0,0,"R"])] + [tuple() for _ in range(len(instructions))]

    # まずは命令を変更せずにinstructionsを実行する
    x,y = 0,0
    d = "R"
    for i in range(len(instructions)):
        if instructions[i] == "F":
            x += d_dict[d][0]
            y += d_dict[d][1]
        elif instructions[i] == "R":
            d = r_dict[d]
        elif instructions[i] == "L":
            d = l_dict[d]
        elif instructions[i] == "B":
            x -= d_dict[d][0]
            y -= d_dict[d][1]
        xy_d[i+1] = (x,y,d)

    # 命令を変更していく
    # 全命令後の座標
    last_x,last_y,last_d = xy_d[-1]
    for i in range(1,len(instructions)+1):
        # i番目の行動後の現在地と向きに対する全命令後の座標の距離(向いている方向に何マス，右に何マス)を計算する
        x,y,d = xy_d[i]
        # 向いている方向に何マス，右に何マス
        f_r = [0,0]
        if d == "R":
            f_r[0] = last_x - x
            f_r[1] = y - last_y
        elif d == "L":
            f_r[0] = x - last_x
            f_r[1] = last_y - y
        elif d == "U":
            f_r[0] = last_y - y
            f_r[1] = last_x - x
        elif d == "D":
            f_r[0] = y - last_y
            f_r[1] = x - last_x
        # print()
        # print(x,y,d)
        # print(f_r)
        # print()
        # 命令を変更していく
        # 行動前の現在地と向き
        x_bef,y_bef,d_bef = xy_d[i-1]
        for j in ["F","R","L","B"]:
            if j == instructions[i-1]:
                continue
            # i番目の命令をjに置き換える
            x_aft,y_aft,d_aft = x_bef,y_bef,d_bef
            # print(j)
            if j == "F":
                x_aft += d_dict[d_bef][0]
                y_aft += d_dict[d_bef][1]
                if d_bef == "R":
                    x_aft += f_r[0]
                    y_aft -= f_r[1]
                elif d_bef == "L":
                    x_aft -= f_r[0]
                    y_aft += f_r[1]
                elif d_bef == "U":
                    x_aft += f_r[1]
                    y_aft += f_r[0]
                elif d_bef == "D":
                    x_aft -= f_r[1]
                    y_aft -= f_r[0]
            elif j == "R":
                if d_bef == "R":
                    x_aft -= f_r[1]
                    y_aft -= f_r[0]
                elif d_bef == "L":
                    x_aft += f_r[1]
                    y_aft += f_r[0]
                elif d_bef == "U":
                    x_aft += f_r[0]
                    y_aft -= f_r[1]
                elif d_bef == "D":
                    x_aft -= f_r[0]
                    y_aft += f_r[1]
            elif j == "L":
                if d_bef == "R":
                    x_aft += f_r[1]
                    y_aft += f_r[0]
                elif d_bef == "L":
                    x_aft -= f_r[1]
                    y_aft -= f_r[0]
                elif d_bef == "U":
                    x_aft -= f_r[0]
                    y_aft += f_r[1]
                elif d_bef == "D":
                    x_aft += f_r[0]
                    y_aft -= f_r[1]
            elif j == "B":
                x_aft -= d_dict[d_bef][0]
                y_aft -= d_dict[d_bef][1]
                if d_bef == "R":
                    x_aft += f_r[0]
                    y_aft -= f_r[1]
                elif d_bef == "L":
                    x_aft -= f_r[0]
                    y_aft += f_r[1]
                elif d_bef == "U":
                    x_aft += f_r[1]
                    y_aft += f_r[0]
                elif d_bef == "D":
                    x_aft -= f_r[1]
                    y_aft -= f_r[0]
            # print(x_aft,y_aft)
            # i番目の命令をjに置き換えた時に，目的地に到達するかどうかを判定する
            if x_aft == tx and y_aft == ty:
                # print("到達する")
                # 到達する場合は，その行動以降をシミュレーションして，障害物にぶつかるかどうかを判定する
                x_now,y_now,d_now = x_bef,y_bef,d_bef
                for k in range(i-1,len(instructions)):
                    instruction = instructions[k]
                    if k == i-1:
                        instruction = j
                    if instruction == "F":
                        if (x_now+d_dict[d_now][0],y_now+d_dict[d_now][1]) not in obstacles:
                            x_now += d_dict[d_now][0]
                            y_now += d_dict[d_now][1]
                        else:
                            break
                    elif instruction == "R":
                        d_now = r_dict[d_now]
                    elif instruction == "L":
                        d_now = l_dict[d_now]
                    elif instruction == "B":
                        if (x_now-d_dict[d_now][0],y_now-d_dict[d_now][1]) not in obstacles:
                            x_now -= d_dict[d_now][0]
                            y_now -= d_dict[d_now][1]
                        else:
                            break
                else:
                    # 障害物にぶつからない場合は，その行動を実行するように命令して終了
                    ind = i
                    if j == "F":
                        inst = "FORWARD"
                    elif j == "R":
                        inst = "TURN RIGHT"
                    elif j == "L":
                        inst = "TURN LEFT"
                    elif j == "B":
                        inst = "BACK"
                    break
        else:
            continue
        break
    return f"Replace instruction {ind} with {inst}"

In [20]:
a = "FFLBBLFLFB"
b = [2, -2]
c = [[-2, -1], [2, 2], [3, 2], [-1, 0], [3, -1], [3, -2], [0, -2], [-1, -2], [2, -3], [1, -1], [3, 3], [1, -3], [4, -1], [0, 1], [-2, 0], [3, 0], [-3, 0], [2, 1], [1, 1], [-1, -1], [-1, 1], [0, -1]]

a="LFLFLB"
b=[-1, 0]
c=[[-1, 1]]

find_correct_path(a,b,c)

'Replace instruction 2 with BACK'

In [ ]:
# 各行動での(行動後の)現在地と向きを記録しておく
# 命令を変更せずに全ての行動が終わった時の現在地を記録しておく

# 各行動について，(行動後の)現在地と向きに対する全命令後の座標の距離(向いている方向に何マス，右に何マス)を計算する
# 各行動を別の行動に変えた時に，目的地に到達するかどうかを判定する
# 到達する場合は，その行動以降をシミュレーションして，障害物にぶつかるかどうかを判定する
# 障害物にぶつからない場合は，その行動を実行するように命令して終了
# 障害物にぶつかる場合，到達しない場合は，次の行動について同様のことを行う

from json import dumps, loads
from typing import List

def find_correct_path(instructions: str, target: List[int], obstacles: List[List[int]]) -> str:
    # 目的地のx,y座標
    tx,ty = target
    # 障害物の座標
    obstacles = set([tuple(o) for o in obstacles])
    # R,L,U,D，各方向を向いている時のFの座標の変化
    d_dict = {"R":(1,0),"L":(-1,0),"U":(0,1),"D":(0,-1)}
    # R,L,U,D，各方向を向いている時のRの向きの変化
    r_dict = {"R":"D","D":"L","L":"U","U":"R"}
    # R,L,U,D，各方向を向いている時のLの向きの変化
    l_dict = {"R":"U","U":"L","L":"D","D":"R"}
    # ind番目の命令をinstに置き換える
    ind = 0
    inst = ""
    # 各行動での(行動後の)現在地と向き (x座標,y座標,方向) xy_d[i]にinstructions[i]を実行するとxy_d[i+1]になる
    xy_d = [tuple([0,0,"R"])] + [tuple() for _ in range(len(instructions))]

    # まずは命令を変更せずにinstructionsを実行する
    x,y = 0,0
    d = "R"
    for i in range(len(instructions)):
        if instructions[i] == "F":
            x += d_dict[d][0]
            y += d_dict[d][1]
        elif instructions[i] == "R":
            d = r_dict[d]
        elif instructions[i] == "L":
            d = l_dict[d]
        elif instructions[i] == "B":
            x -= d_dict[d][0]
            y -= d_dict[d][1]
        xy_d[i+1] = (x,y,d)

    # 命令を変更していく
    # 全命令後の座標
    last_x,last_y,last_d = xy_d[-1]
    # 探索後の状態
    seen = set()
    for i in range(1,len(instructions)+1):
        # i番目の行動後の現在地と向きに対する全命令後の座標の距離(向いている方向に何マス，右に何マス)を計算する
        x,y,d = xy_d[i]
        # 向いている方向に何マス，右に何マス
        f_r = [0,0]
        if d == "R":
            f_r[0] = last_x - x
            f_r[1] = y - last_y
        elif d == "L":
            f_r[0] = x - last_x
            f_r[1] = last_y - y
        elif d == "U":
            f_r[0] = last_y - y
            f_r[1] = last_x - x
        elif d == "D":
            f_r[0] = y - last_y
            f_r[1] = x - last_x
        # print()
        # print(x,y,d)
        # print(f_r)
        # print()
        # 命令を変更していく
        # 行動前の現在地と向き
        x_bef,y_bef,d_bef = xy_d[i-1]
        if (x_bef,y_bef,d_bef,f_r[0],f_r[1]) in seen:
            continue
        seen.add((x_bef,y_bef,d_bef,f_r[0],f_r[1]))
        for j in ["F","R","L","B"]:
            if j == instructions[i-1]:
                continue
            # i番目の命令をjに置き換える
            x_aft,y_aft,d_aft = x_bef,y_bef,d_bef
            # print(j)
            if j == "F":
                x_aft += d_dict[d_bef][0]
                y_aft += d_dict[d_bef][1]
                if d_bef == "R":
                    x_aft += f_r[0]
                    y_aft -= f_r[1]
                elif d_bef == "L":
                    x_aft -= f_r[0]
                    y_aft += f_r[1]
                elif d_bef == "U":
                    x_aft += f_r[1]
                    y_aft += f_r[0]
                elif d_bef == "D":
                    x_aft -= f_r[1]
                    y_aft -= f_r[0]
            elif j == "R":
                if d_bef == "R":
                    x_aft -= f_r[1]
                    y_aft -= f_r[0]
                elif d_bef == "L":
                    x_aft += f_r[1]
                    y_aft += f_r[0]
                elif d_bef == "U":
                    x_aft += f_r[0]
                    y_aft -= f_r[1]
                elif d_bef == "D":
                    x_aft -= f_r[0]
                    y_aft += f_r[1]
            elif j == "L":
                if d_bef == "R":
                    x_aft += f_r[1]
                    y_aft += f_r[0]
                elif d_bef == "L":
                    x_aft -= f_r[1]
                    y_aft -= f_r[0]
                elif d_bef == "U":
                    x_aft -= f_r[0]
                    y_aft += f_r[1]
                elif d_bef == "D":
                    x_aft += f_r[0]
                    y_aft -= f_r[1]
            elif j == "B":
                x_aft -= d_dict[d_bef][0]
                y_aft -= d_dict[d_bef][1]
                if d_bef == "R":
                    x_aft += f_r[0]
                    y_aft -= f_r[1]
                elif d_bef == "L":
                    x_aft -= f_r[0]
                    y_aft += f_r[1]
                elif d_bef == "U":
                    x_aft += f_r[1]
                    y_aft += f_r[0]
                elif d_bef == "D":
                    x_aft -= f_r[1]
                    y_aft -= f_r[0]
            # print(x_aft,y_aft)
            # i番目の命令をjに置き換えた時に，目的地に到達するかどうかを判定する
            if x_aft == tx and y_aft == ty:
                # print("到達する")
                # 到達する場合は，その行動以降をシミュレーションして，障害物にぶつかるかどうかを判定する
                x_now,y_now,d_now = x_bef,y_bef,d_bef
                for k in range(i-1,len(instructions)):
                    instruction = instructions[k]
                    if k == i-1:
                        instruction = j
                    if instruction == "F":
                        if (x_now+d_dict[d_now][0],y_now+d_dict[d_now][1]) not in obstacles:
                            x_now += d_dict[d_now][0]
                            y_now += d_dict[d_now][1]
                        else:
                            break
                    elif instruction == "R":
                        d_now = r_dict[d_now]
                    elif instruction == "L":
                        d_now = l_dict[d_now]
                    elif instruction == "B":
                        if (x_now-d_dict[d_now][0],y_now-d_dict[d_now][1]) not in obstacles:
                            x_now -= d_dict[d_now][0]
                            y_now -= d_dict[d_now][1]
                        else:
                            break
                else:
                    # 障害物にぶつからない場合は，その行動を実行するように命令して終了
                    ind = i
                    if j == "F":
                        inst = "FORWARD"
                    elif j == "R":
                        inst = "TURN RIGHT"
                    elif j == "L":
                        inst = "TURN LEFT"
                    elif j == "B":
                        inst = "BACK"
                    break
        else:
            continue
        break
    return f"Replace instruction {ind} with {inst}"